# Single Unit Properties at Zillow: Predicting Values and Identifying Drivers
***

## Goals

This project has two primary goals:

1) Create a model that will acurrately predict the values of single unit properties that the tax district assesses. 

2) Identify and report the drivers of the values of single unit properties that the tax district assesses.


In [1]:
# Establishing environment

from env import host, user, password

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from prepare import prep_zillow_data

import warnings
warnings.filterwarnings("ignore")

## Acquire
Acquire data from the customers table from the zillow database on the codeup data science database server.
***

Function below creates a URL string that can be used to connect to the data science database

In [2]:
def get_connection(db, user=user, host=host, password=password):
    """
    Function creates a URL that can be used to connect to the data science database.
    """
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

This next function creates a dataframe from the zillow data by using the URL string from the first function and connecting to the zillow table in the data science database.

In [3]:
def get_zillow_data():
    """
    Function connects to the data science database and returns a data frame containing Zillow data for houses whose last transaction was in May through June 2017.
    """
    # SQL query string
    sql_query = "SELECT * FROM properties_2017 JOIN predictions_2017 on predictions_2017.parcelid = properties_2017.parcelid WHERE unitcnt = 1 AND transactiondate BETWEEN '2017-05-01' AND '2017-06-30'"
    # creates dataframe using data from DS database
    df = pd.read_sql(sql_query, get_connection('zillow'))
    # returns DF 
    return df

We will now test our function to ensure it works.

In [4]:
# creating dataframe using data acquired from function
df = get_zillow_data()

# previewing data to ensure function operates correctly
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,parcelid,logerror,transactiondate
0,2061546,11289917,1.0,None,None,2.0,3.0,NaN,6.0,2.0,...,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,11289917,-0.362001,2017-06-23
1,1834372,11705026,NaN,None,None,1.0,2.0,NaN,6.0,1.0,...,2016.0,23624.0,543.69,None,NaN,6.037232e+13,1772,11705026,-0.146056,2017-06-30
2,2121349,11389003,NaN,None,None,2.0,3.0,NaN,6.0,2.0,...,2016.0,449000.0,7673.19,None,NaN,6.037703e+13,3273,11389003,-0.325393,2017-06-01
3,2093710,11967869,NaN,None,None,1.0,2.0,NaN,5.0,1.0,...,2016.0,191821.0,3267.47,None,NaN,6.037185e+13,3429,11967869,-0.005566,2017-06-29
4,1288537,12035176,NaN,None,None,1.0,1.0,NaN,11.0,1.0,...,2016.0,146810.0,2926.19,None,NaN,6.037189e+13,5444,12035176,-0.114435,2017-05-24


### Acquire Takeaways

- Acquiring data from zillow database on the codeup data science database server via get_zillow_data function
- Both functions from this section are available in the __acquire.py__ file deliverable

## Prepare

Prepare, tidy, and clean the data so it can be explored and analyzed.
***

In [5]:
# displaying column names and null value counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13309 entries, 0 to 13308
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            13309 non-null  int64  
 1   parcelid                      13309 non-null  int64  
 2   airconditioningtypeid         6226 non-null   float64
 3   architecturalstyletypeid      0 non-null      object 
 4   basementsqft                  0 non-null      object 
 5   bathroomcnt                   13309 non-null  float64
 6   bedroomcnt                    13309 non-null  float64
 7   buildingclasstypeid           2 non-null      float64
 8   buildingqualitytypeid         13162 non-null  float64
 9   calculatedbathnbr             13296 non-null  float64
 10  decktypeid                    0 non-null      object 
 11  finishedfloor1squarefeet      0 non-null      object 
 12  calculatedfinishedsquarefeet  13308 non-null  float64
 13  f

Dropping columns that meet any of the following criterion:
* more than 2,000 null values 
* do not contain relevant data for predicting property value 
    - examples include irrelevant IDs, fips, etc.
* taxamount, structuretaxvaluedollarcnt, landtaxvaluedollarcnt 
    - per project specifications we cannot use these columns
        - "for prediction purposes, we won't know tax amount until we know tax value"
        - confirmed by Data Science senior team member John Salas

In [6]:
# .nunique shows us how many unique values are held in each column
df.nunique()

id                          13308
parcelid                    13308
airconditioningtypeid           1
architecturalstyletypeid        0
basementsqft                    0
                            ...  
censustractandblock         10177
id                          13309
parcelid                    13308
logerror                    13213
transactiondate                56
Length: 63, dtype: int64

Dropping columns that meet any of the following criterion:
- categorical columns with more than 5 unique values 
    - it will be cumbersome to create a boolean value for each
- columns with only 1 value as 
    - our model won't be able to make any meaningful distinctions with them

While viewing data in SQL we noted several columns that appear to have identical data to one another. We will confirm if this is the case.

In [7]:
# comparing each pair of columns we suspected are duplicates
# the resulting variables tell us how many rows differ between each
sqft_columns_diff = (df.finishedsquarefeet12 != df.calculatedfinishedsquarefeet).sum()
bathroom_count_diff = (df.calculatedbathnbr != df.bathroomcnt).sum()
bathroom_count_diff_alt = (df.fullbathcnt != df.bathroomcnt).sum()

print(f'Number of different values between finishedsquarefeet12 and calculatedfinishedsquarefeet: {sqft_columns_diff}')
print(f'Number of different values between calculatedbathnbr and bathroomcnt: {bathroom_count_diff}')
print(f'Number of different values between fullbathcnt and bathroomcnt: {bathroom_count_diff_alt}')

Number of different values between finishedsquarefeet12 and calculatedfinishedsquarefeet: 13
Number of different values between calculatedbathnbr and bathroomcnt: 13
Number of different values between fullbathcnt and bathroomcnt: 13


Dropping the following columns because with the exception of 13 rows, they are duplicates:
- finishedsquarefeet12 (near-duplicate of calculatedfinishedsquarefeet)
- calculatedbathnbr (near-duplicate of bathroomcnt)
- bathroomcnt (near-duplicate of bathroomcnt)

In [8]:
# Dropping columns with more than 2000 rows missing
df = df.drop(columns=['airconditioningtypeid', 'architecturalstyletypeid', 'basementsqft','yardbuildingsqft26', 'buildingclasstypeid', 'decktypeid', 'finishedfloor1squarefeet', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'regionidneighborhood', 'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid', 'yardbuildingsqft17', 'numberofstories', 'fireplaceflag', 'taxdelinquencyflag', 'taxdelinquencyyear'], axis=1)

# Dropping columns with high amount of duplicate values
df = df.drop(columns=['finishedsquarefeet12', 'calculatedbathnbr', 'fullbathcnt'], axis=1)

# Dropping columns with irrelevant data
df = df.drop(columns=['id', 'fips', 'parcelid', 'latitude', 'longitude', 'rawcensustractandblock', 'censustractandblock', 'heatingorsystemtypeid', 'regionidcounty', 'logerror' ], axis=1)

# Dropping categorical columns with more than 5 unique values
df = df.drop(columns=['propertycountylandusecode', 'propertylandusetypeid','buildingqualitytypeid', 'propertyzoningdesc', 'regionidcity', 'regionidzip', 'yearbuilt', 'transactiondate' ], axis=1)

# Dropping columns with only 1 unique value
df = df.drop(columns=['roomcnt', 'unitcnt', 'assessmentyear'], axis=1)

# Dropping columns that are prohibited from use
df = df.drop(columns=['taxamount', 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt'], axis=1)

- All columns that we specified for removal have been dropped. We'll now take another look at the data with .info for further changes needed.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13309 entries, 0 to 13308
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   13309 non-null  float64
 1   bedroomcnt                    13309 non-null  float64
 2   calculatedfinishedsquarefeet  13308 non-null  float64
 3   lotsizesquarefeet             12976 non-null  float64
 4   taxvaluedollarcnt             13309 non-null  float64
dtypes: float64(5)
memory usage: 520.0 KB


We will change the names of our remaining columns to make them easier to understand
- Rename bathroomcnt to bathroom_count
- Rename bedroomcnt to bedroom_count
- Rename calculatedfinishedsquarefeet to unit_sq_feet
- Rename lotsizesquarefeet to lot_size_sq_feet
- Rename taxvaluedollarcnt to tax_dollar_value

lotsizesquarefeet has 333 missing values
- Out of 13309 rows, this is less than 3% of the total so we'll remove them along with the one null value row from calculatedfinishedsquarefeet

In [10]:
# using head to examine raw values of columns
df.head(10)

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,lotsizesquarefeet,taxvaluedollarcnt
0,2.0,3.0,1458.0,8284.0,136104.0
1,1.0,2.0,1421.0,6707.0,35606.0
2,2.0,3.0,1650.0,7300.0,614000.0
3,1.0,2.0,693.0,2908.0,274237.0
4,1.0,1.0,812.0,93226.0,245906.0
5,2.0,3.0,1108.0,6050.0,486866.0
6,2.0,3.0,1947.0,6404.0,388478.0
7,1.0,2.0,1061.0,4800.0,31839.0
8,2.0,3.0,1632.0,6040.0,499402.0
9,3.0,3.0,2768.0,34309.0,151340.0


- No binary values
- Various units of measurement (dollars, feet, bedrooms, bathrooms)
- All columns contain numerical values

For these reasons we will need to scale all of our columns except taxvaluedollarcnt (since it's our target variable and target variables should not be scaled).

In [11]:
# creating data frames that we'll need to progress through project via prep_zillow function from prepare.py file
train, y_train, y_validate, y_test, X_train_scaled, X_train, X_validate, X_test, X_validate_scaled, X_test_scaled = prep_zillow_data('taxvaluedollarcnt')

In [12]:
# sampling dataframe to confirm function ran successfully
X_train_scaled.head()

,unit_sq_feet,bedroom_count,bathroom_count,lot_size_sq_feet
0,0.078248,0.2,0.3,0.003961
1,0.073514,0.3,0.2,0.002864
2,0.078117,0.4,0.2,0.003505
3,0.065952,0.3,0.2,0.002623
4,0.142688,0.4,0.3,0.006710


## Prepare Takeaways

Removing columns that meet any of the following criterion:
- More than 2,000 row missing
- Duplicates or near-duplicates of other columns
- Contain data that's irrelevant to our project
- Categorical with more than 5 unique values
- Contain only 1 unique column
- Are prohibited from being used per project spefications

Rename remaining columns for readability and understanding

Drop rows with missing values since this accounts for less than 3% of all rows

Scale every non-target variable column. (ie. everything except taxvaluedollarcnt)

Split all data into train, validate and test samples

All of these changes will be done via the __prep_zillow_data__ function within the __prepare.py__ file deliverable